# AirBnB Project for  **Project: Write A Data Science Blog Post**

### 0.1  Key Steps for Project

Feel free to be creative with your solutions, but do follow the CRISP-DM process in finding your solutions.

1) Pick a dataset.
   I chose the AirBnnB dataset.  Because why not..

2) Pose at least three questions related to business or real-world applications of how the data could be used.

3) Create a Jupyter Notebook, using any associated packages you'd like, to:

    Prepare data:
        Gather necessary data to answer your questions
        Handle categorical and missing data
        Provide insight into the methods you chose and why you chose them

    Analyze, Model, and Visualize
        Provide a clear connection between your business questions and how the data answers them.

4) Communicate your business insights:

    Create a Github repository to share your code and data wrangling/modeling techniques, with a technical audience in mind
    Create a blog post to share your questions and insights with a non-technical audience

Your deliverables will be a Github repo and a blog post. Use the rubric here to assist in successfully completing this project!

## 0.2 [Rubric](https://review.udacity.com/#!/rubrics/1507/view)

#### Code Functionality and Readability
* Code is readable (uses good coding practices - PEP8) 
* Code is functional.
* Write code that is well documented and uses functions and classes as necessary.

#### Data
* Project follows the CRISP-DM Process while analyzing their data.
* Proper handling of categorical and missing values in the dataset.
* Categorical variables are handled appropriately for machine learning models (if models are created). 

#### Analysis, Modeling, Visualization
* There are 3-5 business questions answered.
	
#### Github Repository
* Student must publish their code in a public Github repository.
	
#### Blog Post
* Communicate their findings with stakeholders.
* There should be an intriguing title and image related to the project.
* The body of the post has paragraphs that are broken up by appropriate white space and images.
* Each question has a clearly communicated solution.

##  0.3  CRISP-DM
### 0.3.1 Business Understanding/Data Understanding
          AirBnB is an online marketplace for vacation/temporary houseing rentals.  Thier members/hosts own the property and rent via the  AirBnB marketplace.
          
          The data provides was provided each from Seattle and Bostom
          * listing.csv
          * calendar.csv
          * reviews.csv
          
          How to best position your property to make the most revenue?
          1) What time of year is best to make your property availible to get the most money?
              Summer months for most properties, but there seem to bes some very high outliers around the holidays...
          2) Know your capacity?   What neighborhoods/property type/property attributes sell better?
   
          3) What do you need in your profile to garuntee your property is reviewed and reviewed high
          
          
### 0.3.2 Data Preparation
        1) Imported listing and calendar
        #clean_cal() and clean_listing()
        2) Corrected and adjusted datatypes  
        3) Took some columns and made them into booleans (presnence or absence of awnser)
        
        
     
#### 0.3.2.1 Cleaning Data
        Price is our response variable
        1) removed all rows where price was NA
        2) Removed a column that was mostly NAs
        3) removed a few rows of the remaining NA
        
        
### 0.3.3 Modeling
### 0.3.4 Evaluation
### 0.3.5 Deployment

## 1.1 Header

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import os.path as op

#stats
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

import ast
import math
import os
from datetime import date

#Graphing
import seaborn as sns
import matplotlib.pyplot as plt
import helper as h

#Data modeling
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.decomposition import pca

In [ ]:
PATH = os.getcwd()+"\All Data"
PATH

In [ ]:
os.listdir(PATH)

In [ ]:
def clean_listings(df):
    ''' Return a cleaned dataframe derived from listing.csv file

    1) Fixes percentage & currency strings to float
    2) Fixes datatype of oject to categories were appropraite
    3) Fixes bool strings to bool cols
    2) Fixes Datetime cols -> datetime format
    3) There are two columns that have list containing strings, convert to one hot encoded columns 
    4) 
    5) 

    Parameters
    -------
    df:  Pandas DataFrame with an already imported lsiting.csv

    '''

    ## Clean percentage strings to float values
    pct_col = ['host_acceptance_rate','host_response_rate']
    for pc in pct_col:
        df[pc] = df[pc].str.strip("%")
        df[pc] = df[pc].astype('float')
        df[pc] = df[pc].apply(lambda x: x/100)
        #df[pc] = df[pc].map('{:,.2%}'.format)
        
    ## Clean dollar strings to value

    dol_col = ['price','weekly_price','monthly_price','security_deposit','cleaning_fee','extra_people']
    for dol in dol_col:
        df[dol] = df[dol].str.replace('$','')
        df[dol] = df[dol].str.replace(',','')
        df[dol] = df[dol].astype('float')
        #df[dol] = df[dol].map('${:,.2f}'.format)
        
    ## Change type to category
    cat_col = ['host_response_time','host_location','host_neighbourhood','neighbourhood',
               'neighbourhood_cleansed','neighbourhood_group_cleansed','city','state','zipcode',
              'market','smart_location','country_code','country','property_type','room_type',
              'calendar_updated','jurisdiction_names','cancellation_policy','bed_type']
    
    for cc in cat_col:
        df[cc] = df[cc].astype('category')
  

    ## Fix Boolean Columns
    bool_col = ['host_is_superhost','host_has_profile_pic','host_identity_verified',
                'is_location_exact','has_availability','requires_license','instant_bookable',
               'require_guest_profile_picture','require_guest_phone_verification']
    for bc in bool_col:
        df[bc] = df[bc].replace({'t': True,'f':False})
        df[bc] = df[bc].astype(bool)

    ## Fix Datetime columns
    dt_col = ['last_scraped','host_since','calendar_last_scraped','first_review','last_review']
    for dt in dt_col:
        df[dt] = pd.to_datetime(df[dt])

    ## Fix list column
    ## The following code transforms column 'host_verification' to a usable matrix of 
    ##     one hot encoding the contained communicaiton methods      
    
    df2 = pd.DataFrame(df['host_verifications'].apply(lambda x:ast.literal_eval(x)))  # string to list #
    df3 = df2.host_verifications.apply(pd.Series)                                   # list -> series across columns #
    df2 = df2.merge(df3, right_index=True, left_index=True)
    df2 = df2.reset_index().melt(id_vars=['id','host_verifications'],value_name = 'host_sm_ver')
    df2 = df2.pivot_table(values='variable',columns='host_sm_ver',index='id',aggfunc='count',fill_value=0)
    df2 = df2.add_prefix('hv::')
    df = df.merge(df2,left_index=True, right_index=True)

    
    
    df4 = pd.DataFrame(df_listing['amenities'].apply(lambda x:
                           x.replace('\"','').replace('{','').replace('}','').split(',')))
    df5 = df4.iloc[:,0].apply(pd.Series)
    df4 = df4.merge(df5,left_index=True,right_index=True)
    df4 = df4.reset_index().melt(id_vars=['id','amenities'],value_name = 'amms')
    df4 = df4.pivot_table(values='variable',columns='amms',index='id',aggfunc='count',fill_value=0)
    df4 = df4.add_prefix('amm::')
    df = df.merge(df4,left_index=True, right_index=True)
    
    ## Remaining string columns, only interested in the presence or absence.   Replace with boolean. 
    
    
    str_cols = ['summary', 'space', 'neighborhood_overview', 'notes', 'transit',
   'thumbnail_url', 'medium_url', 'xl_picture_url', 'host_about']
    
    def replace_str(x):
        if type(x) == float:
            if  math.isnan(x):
                return False
        else: return True
    
    df_listed_attributes = pd.DataFrame()
    for col in str_cols:
        df_listed_attributes[col] = df[col].apply(lambda x: replace_str(x))
        df_listed_attributes[col] = df_listed_attributes[col].astype('bool')
    
    df_listed_attributes = df_listed_attributes.add_prefix('attrib::')
    df = df.merge(df_listed_attributes,left_index=True,right_index=True)
    df = df.drop(columns=str_cols)
    
    
    ## Drop Columns
    '''Reasons
    All N/A: licence
    No N/A: listing_url
    onehot: host_verifications
    am
       
    '''
    drop_cols = ['license','host_verifications','amenities']  
    df = df.drop(columns=drop_cols)
    return df


In [ ]:
def clean_cal(df):
    ''' Return a cleaned dataframe derived from listing.csv file

    1) Fixes Datetime cols -> datetime format
    2) Fixes percentage strings to float
    3) Fixes bool strings to bool cols
    4) Fixes datatype of oject to categories were appropraite
    5) 

    Parameters
    -------
    df:  Pandas DataFrame with an already imported lsiting.csv

    '''
    dol_col= ['price']
    for dol in dol_col:
        df[dol] = df[dol].str.replace('$','')
        df[dol] = df[dol].str.replace(',','')
        df[dol] = df[dol].astype('float')
        
    ## Fix Datetime columns
    dt_col = ['date']
    for dt in dt_col:
        df[dt] = pd.to_datetime(df[dt])

        
    ## Expand Datetime
    df['week_num'] = df['date'].apply(lambda x: date.isocalendar(x)[1])
    df['month'] = df['date'].apply(lambda x: x.month)
    df['year'] = df['date'].apply(lambda x: x.year)
    
    ## Fix Bool Columns

    bool_col = ['available']
    for bc in bool_col:
        df[bc] = df[bc].replace({'t': True,'f':False})
        df[bc] = df[bc].astype(bool)

    
    
    
    return df

In [ ]:
df_listing = pd.read_csv(PATH+'\s_listings.csv',index_col=0)

In [ ]:
df_cal = pd.read_csv(PATH+'\s_calendar.csv',index_col=0)

In [ ]:
df_listing = clean_listings(df_listing)

In [ ]:
df_cal = clean_cal(df_cal)

### Q1.What time of year is best to make your property availible to get the most money?

In [ ]:
pd.DataFrame(df_cal.groupby(['month'])['price'].mean()).plot()

import matplotlib.pyplot as plt

In [ ]:
ax1 = sns.lineplot(x=df_cal['week_num'],y=df_cal['price'])

In [ ]:
ax1 = sns.boxplot(y=df_cal['price'],x=df_cal['month'],)

In [ ]:
ax1 = sns.boxplot(y=df_cal['price'],x=df_cal['month'],showfliers=False)

In [ ]:
mod = ols('price ~ month',data=df_cal).fit()
aov_table = sm.stats.anova_lm(mod, typ=2)
print(aov_table)

In [ ]:
#Create a boxplot
df_cal.boxplot('price', by='month', figsize=(6, 4),showfliers=False)

## END OF QUESTION 1

Question 2:  Know your capacity?   Waht neighborhoods/property type/property attributes sell better?

In [ ]:
def compile_and_clean_attr_data(df_list,df_cal):
    
    #Using the average list prices in df_cal['price'] vs the current listing price df_list['price']
    df_avg_price = pd.DataFrame(df_cal.groupby(by=['listing_id'])['price'].mean())
    df_list = df_list.drop(columns='price')
    
    
    df = df_avg_price.merge(df_list,left_index=True,right_index=True)
    
    #Strip Xs down to the property/physical attributes of the property
    ammenities = list(df.filter(like='amm').columns.values)
    physical_attr_loc = ['price','bathrooms','bedrooms','beds','neighbourhood_cleansed',
                     'neighbourhood_group_cleansed','property_type','room_type','bed_type',
                    'accommodates','guests_included'] + ammenities
    
    df = df[physical_attr_loc]
    ##Clean Nas 
    
    # Drop all rows without a y (price)
    df = df.dropna(axis=0,subset=['price'])
    
    # Dropping rows b/c The following had a low number of n/a rows and would be difficult to impute, 
    df = df.drop(index=df.loc[df['bedrooms'].isna(),:].index )
    df = df.drop(index=df.loc[df['property_type'].isna(),:].index )
    df = df.drop(index=df.loc[df['bathrooms'].isna(),:].index )
    
    
    #Creating a log-transformed price column to 
    df.insert(column='price_log',loc=1,value=np.nan)
    df['price_log'] = df['price'].apply(lambda x: np.log(x))
    
    # One hot encode the categorical columns
    cat_col = list(df.select_dtypes(include='category').columns.values)
    df = df.merge(pd.get_dummies(df[cat_col],prefix_sep='::'),right_index=True, left_index=True)
    df = df.drop(columns=cat_col)
    
    #Standardize
    std_col = ['price','price_log']
    
    #Normalize
    norm_col = ['accommodates','guests_included','bathrooms','bedrooms','beds']
    
    return df

In [ ]:
df_q2 = compile_and_clean_attr_data(df_listing,df_cal)

#### Standardize and Normalize


In [ ]:
#Standardize
std_col = ['price','price_log']
blah = StandardScaler()

df_q2[std_col] = blah.fit_transform(df_q2[std_col])

#Normalize
norm_col = ['accommodates','guests_included','bathrooms','bedrooms','beds']

df_q2[norm_col] = normalize(df_q2[norm_col])

#### Linear Model

In [ ]:
df_q2_X = df_q2.drop(columns=['price','price_log'])
df_q2_y1 = df_q2['price']
df_q2_y2 = df_q2['price_log']

q2_model = LinearRegression()

q2_log_model = LinearRegression()

q2_X_train, q2_X_test, q2_y1_train, q2_y1_test, q2_y2_train, q2_y2_test = train_test_split(df_q2_X,df_q2_y1,df_q2_y2,test_size = 0.3,random_state=72)

q2_model.fit(q2_X_train,q2_y1_train)

q2_log_model.fit(q2_X_train,q2_y2_train)

q2_y1_predict = q2_model.predict(q2_X_test)

q2_y2_predict = q2_log_model.predict(q2_X_test)

In [ ]:
r2_score(q2_y1_test,q2_y1_predict)

In [ ]:
r2_score(q2_y2_test,q2_y2_predict)

In [ ]:
coefs_df = pd.DataFrame()
coefs_df['est_int'] = q2_X_train.columns
coefs_df['coefs'] = q2_model.coef_
coefs_df['abs_coefs'] = np.abs(q2_model.coef_)
coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
coefs_df[0:15]

In [ ]:
coefs_df['type'] = coefs_df['est_int'].apply(lambda x: x.split(':')[0])

In [ ]:
coefs_df.groupby(by='type')['abs_coefs'].mean().sort_values(ascending=False).head(6)

In [ ]:
df_room_type = df_cal.merge(df_listing[['room_type']],left_index=True,right_index=True)

In [ ]:
sns.boxplot(x=df_room_type['room_type'],y=df_room_type['price'],showfliers=False)

In [ ]:
#### PCA MODEL

In [ ]:
model_1 = pca.PCA(n_components=35)

q2_X_pca = model_1.fit_transform(df_q2_X)

model_1.explained_variance_ratio_.sum()

In [ ]:
h.scree_plot(model_1)

In [ ]:
df_q2_X_pca = pd.DataFrame(q2_X_pca)

X_train_pca, X_test_pca, y_train, y_test = train_test_split(df_q2_X_pca,df_q2_y1,test_size = 0.3,random_state=72)

lmodel_pca = LinearRegression()

lmodel_pca.fit(X_train_pca,y_train)

y_predict_pca = lmodel_pca.predict(X_test_pca)

r2_score(y_test,y_predict_pca)

In [ ]:
pca_results = h.pca_results_rng(df_q2_X,model_1,[1,25])
pca_results.head()

In [ ]:
pca_results = pca_results.drop(columns='Explained Variance')

In [ ]:
pca_coefs_df = pd.DataFrame()
pca_coefs_df['est_int'] = X_train_pca.columns
pca_coefs_df['coefs'] = lmodel_pca.coef_
pca_coefs_df['abs_coefs'] = np.abs(lmodel_pca.coef_)
pca_coefs_df = pca_coefs_df.sort_values('abs_coefs', ascending=False)
pca_coefs_df[0:6]

In [ ]:
print(pca_results.loc['Dimension 2',:].sort_values(ascending=False)[0:10])
print(pca_results.loc['Dimension 2',:].sort_values(ascending=True)[0:6])

In [ ]:
print(pca_results.loc['Dimension 24',:].sort_values(ascending=False)[0:10])
print(pca_results.loc['Dimension 24',:].sort_values(ascending=True)[0:6])

In [ ]:
print(pca_results.loc['Dimension 14',:].sort_values(ascending=False)[0:10])
print(pca_results.loc['Dimension 14',:].sort_values(ascending=True)[0:6])

#### COMPARE NEIGHBOORHOODS

In [ ]:
hot_neighboorhoods=['Magnolia','Downtown','Queen Anne','West Seattle']

def hotchecks(x,list):
    if x in hot_neighboorhoods:
        return True
    else:
        return False

df_neigh = df_cal.merge(df_listing[['neighbourhood_group_cleansed']],left_index=True,right_index=True)

df_neigh.insert(loc = df_neigh.shape[1],column='hot_neigh',value=False,)

df_neigh['hot_neigh'] = df_neigh['neighbourhood_group_cleansed'].apply(lambda x: hotchecks(x,hot_neighboorhoods))

month = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}

df_neigh['month'] = df_neigh['month'].apply(lambda x: month[x])

In [ ]:
f,ax = plt.subplots(1,2,sharey=True)
f.set_figwidth(15)

sns.set_style('darkgrid')
sns.barplot(x=df_neigh['month'],y=df_neigh['price'],hue=df_neigh['hot_neigh'],ax=ax[1])
sns.barplot(x=df_neigh['month'],y=df_neigh['price'],color='cyan',ax=ax[0])

f.subplots_adjust(wspace=0.01, hspace=0)

ax[1].set(ylim=(0, 220))
ax[0].set(ylim=(0, 220))

ax[0].set_ylabel('Price ($)')
ax[1].set_ylabel('')
ax[0].set_xlabel('')
ax[1].set_xlabel('')

ax[1].set_title('Average Price by Month across hot and not neighboorhoods')
ax[0].set_title('Average Price by Month (All)')
f.show()

In [ ]:
df_neigh.groupby(by='neighbourhood_group_cleansed')['price'].mean()